In [1]:
import os
from PIL import Image

def create_low_res_images(input_dir, output_dir, scale):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for filename in os.listdir(input_dir):
        if filename.endswith('.png') or filename.endswith('.jpg'):
            img = Image.open(os.path.join(input_dir, filename))
            # 고해상도에서 저해상도 이미지 생성
            lr_img = img.resize((img.width // scale, img.height // scale), Image.BICUBIC)
            # 생성된 저해상도 이미지를 저장
            lr_img.save(os.path.join(output_dir, filename))

# 경로 설정
train_hr_dir = r"C:\Users\kowm6\Desktop\SRdataset\DIV2K\DIV2K_train_HR"
valid_hr_dir = r"C:\Users\kowm6\Desktop\SRdataset\DIV2K\DIV2K_valid_HR"

train_lr_dir = r"C:\Users\kowm6\Desktop\SRdataset\DIV2K\DIV2K_train_LR"
valid_lr_dir = r"C:\Users\kowm6\Desktop\SRdataset\DIV2K\DIV2K_valid_LR"

# 배율 설정 (예: ×2)
scale = 2

# 저해상도 이미지 생성
create_low_res_images(train_hr_dir, train_lr_dir, scale)
create_low_res_images(valid_hr_dir, valid_lr_dir, scale)


In [ ]:
import os
from PIL import Image
from torch.utils.data import Dataset
import torchvision.transforms as transforms

class SRDataset(Dataset):
    def __init__(self, hr_dir, lr_dir, transform=None):
        self.hr_dir = hr_dir
        self.lr_dir = lr_dir
        self.hr_filenames = sorted(os.listdir(hr_dir))
        self.lr_filenames = sorted(os.listdir(lr_dir))
        self.transform = transform

    def __len__(self):
        return len(self.hr_filenames)

    def __getitem__(self, idx):
        hr_path = os.path.join(self.hr_dir, self.hr_filenames[idx])
        lr_path = os.path.join(self.lr_dir, self.lr_filenames[idx])

        hr_img = Image.open(hr_path)
        lr_img = Image.open(lr_path)

        if self.transform:
            hr_img = self.transform(hr_img)
            lr_img = self.transform(lr_img)

        return lr_img, hr_img

# 데이터 경로
train_hr_dir = r"C:\Users\kowm6\Desktop\SRdataset\DIV2K\DIV2K_train_HR"
train_lr_dir = r"C:\Users\kowm6\Desktop\SRdataset\DIV2K\DIV2K_train_LR"
valid_hr_dir = r"C:\Users\kowm6\Desktop\SRdataset\DIV2K\DIV2K_valid_HR"
valid_lr_dir = r"C:\Users\kowm6\Desktop\SRdataset\DIV2K\DIV2K_valid_LR"

# 데이터 변환
transform = transforms.Compose([
    transforms.ToTensor(),  # Tensor로 변환
])

# Dataset 및 DataLoader
train_dataset = SRDataset(train_hr_dir, train_lr_dir, transform)
valid_dataset = SRDataset(valid_hr_dir, valid_lr_dir, transform)

from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=16, shuffle=False)


In [2]:
import os
from PIL import Image
from torch.utils.data import Dataset
import torchvision.transforms as transforms

class SRDataset(Dataset):
    def __init__(self, hr_dir, lr_dir, transform=None):
        self.hr_dir = hr_dir
        self.lr_dir = lr_dir
        self.hr_filenames = sorted(os.listdir(hr_dir))
        self.lr_filenames = sorted(os.listdir(lr_dir))
        self.transform = transform

    def __len__(self):
        return len(self.hr_filenames)

    def __getitem__(self, idx):
        hr_path = os.path.join(self.hr_dir, self.hr_filenames[idx])
        lr_path = os.path.join(self.lr_dir, self.lr_filenames[idx])

        hr_img = Image.open(hr_path)
        lr_img = Image.open(lr_path)

        if self.transform:
            hr_img = self.transform(hr_img)
            lr_img = self.transform(lr_img)

        return lr_img, hr_img

# 데이터 경로
train_hr_dir = r"C:\Users\kowm6\Desktop\SRdataset\DIV2K\DIV2K_train_HR"
train_lr_dir = r"C:\Users\kowm6\Desktop\SRdataset\DIV2K\DIV2K_train_LR"
valid_hr_dir = r"C:\Users\kowm6\Desktop\SRdataset\DIV2K\DIV2K_valid_HR"
valid_lr_dir = r"C:\Users\kowm6\Desktop\SRdataset\DIV2K\DIV2K_valid_LR"

# 데이터 변환
transform = transforms.Compose([
    transforms.ToTensor(),  # Tensor로 변환
])

# Dataset 및 DataLoader
train_dataset = SRDataset(train_hr_dir, train_lr_dir, transform)
valid_dataset = SRDataset(valid_hr_dir, valid_lr_dir, transform)

from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=16, shuffle=False)


In [5]:
import os
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision.transforms import functional as F

# 1. SRDataset 클래스 정의
class SRDataset(Dataset):
    def __init__(self, hr_dir, lr_dir, transform=None, target_size=(256, 256)):
        """
        고해상도(HR) 및 저해상도(LR) 이미지 데이터셋.

        Args:
            hr_dir (str): 고해상도 이미지 경로.
            lr_dir (str): 저해상도 이미지 경로.
            transform (callable, optional): 이미지에 적용할 변환 함수.
            target_size (tuple, optional): 리사이즈할 목표 크기.
        """
        self.hr_dir = hr_dir
        self.lr_dir = lr_dir
        self.hr_filenames = sorted(os.listdir(hr_dir))
        self.lr_filenames = sorted(os.listdir(lr_dir))
        self.transform = transform
        self.target_size = target_size  # 고정 크기

    def __len__(self):
        return len(self.hr_filenames)

    def __getitem__(self, idx):
        hr_path = os.path.join(self.hr_dir, self.hr_filenames[idx])
        lr_path = os.path.join(self.lr_dir, self.lr_filenames[idx])

        hr_img = Image.open(hr_path).convert("RGB")
        lr_img = Image.open(lr_path).convert("RGB")

        # 고정 크기로 리사이즈
        hr_img = F.resize(hr_img, self.target_size)
        lr_img = F.resize(lr_img, self.target_size)

        if self.transform:
            hr_img = self.transform(hr_img)
            lr_img = self.transform(lr_img)

        return lr_img, hr_img


# 2. 모델 정의 (SRCNN 예제)
class SRCNN(nn.Module):
    def __init__(self):
        super(SRCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=9, padding=4)
        self.conv2 = nn.Conv2d(64, 32, kernel_size=5, padding=2)
        self.conv3 = nn.Conv2d(32, 3, kernel_size=5, padding=2)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.conv3(x)
        return x


# 3. 데이터 경로 설정
train_hr_dir = r"C:\Users\kowm6\Desktop\SRdataset\DIV2K\DIV2K_train_HR"
train_lr_dir = r"C:\Users\kowm6\Desktop\SRdataset\DIV2K\DIV2K_train_LR"
valid_hr_dir = r"C:\Users\kowm6\Desktop\SRdataset\DIV2K\DIV2K_valid_HR"
valid_lr_dir = r"C:\Users\kowm6\Desktop\SRdataset\DIV2K\DIV2K_valid_LR"

# 데이터 변환 설정
transform = transforms.Compose([
    transforms.ToTensor(),  # 이미지를 Tensor로 변환
])

# Dataset 및 DataLoader 설정
train_dataset = SRDataset(train_hr_dir, train_lr_dir, transform, target_size=(256, 256))
valid_dataset = SRDataset(valid_hr_dir, valid_lr_dir, transform, target_size=(256, 256))

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=16, shuffle=False)





In [6]:
# 4. 학습 준비
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SRCNN().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)


In [7]:
# 5. 학습 루프
num_epochs = 1  # 에포크 수 설정
for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0

    for lr_imgs, hr_imgs in train_loader:
        lr_imgs, hr_imgs = lr_imgs.to(device), hr_imgs.to(device)

        # 모델 예측
        preds = model(lr_imgs)

        # 손실 계산
        loss = criterion(preds, hr_imgs)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss / len(train_loader):.4f}")




Epoch 1/1, Loss: 0.0622


In [8]:
# 6. 검증
model.eval()
with torch.no_grad():
    valid_loss = 0
    for lr_imgs, hr_imgs in valid_loader:
        lr_imgs, hr_imgs = lr_imgs.to(device), hr_imgs.to(device)
        preds = model(lr_imgs)
        loss = criterion(preds, hr_imgs)
        valid_loss += loss.item()

    print(f"Validation Loss: {valid_loss / len(valid_loader):.4f}")

Validation Loss: 0.0156


In [16]:
# 최적 가중치 저장 경로와 초기값 설정
save_path = r"C:\Users\kowm6\Desktop\sr\best_model.pth"
best_valid_loss = float('inf')  # 초기값 설정 (무한대로 초기화)

# 학습 루프
num_epochs = 2
for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0

    for lr_imgs, hr_imgs in train_loader:
        lr_imgs, hr_imgs = lr_imgs.to(device), hr_imgs.to(device)
        preds = model(lr_imgs)
        loss = criterion(preds, hr_imgs)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    # 검증
    model.eval()
    valid_loss = 0
    with torch.no_grad():
        for lr_imgs, hr_imgs in valid_loader:
            lr_imgs, hr_imgs = lr_imgs.to(device), hr_imgs.to(device)
            preds = model(lr_imgs)
            loss = criterion(preds, hr_imgs)
            valid_loss += loss.item()

    valid_loss /= len(valid_loader)
    print(f"Epoch {epoch + 1}/{num_epochs}, Train Loss: {epoch_loss / len(train_loader):.4f}, Validation Loss: {valid_loss:.4f}")

    # 최적 가중치 저장
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), save_path)
        print(f"Best model saved at epoch {epoch + 1}")


Epoch 1/2, Train Loss: 0.0042, Validation Loss: 0.0037
Best model saved at epoch 1
Epoch 2/2, Train Loss: 0.0027, Validation Loss: 0.0026
Best model saved at epoch 2


In [ ]:
# 6. 테스트
test_hr_dir = "C:/Users/kowm6/Desktop/SRdataset/BSD100/BSD100/image_SRF_2/img_001_SRF_2_HR.png"
test_lr_dir = "C:/Users/kowm6/Desktop/SRdataset/BSD100/BSD100/image_SRF_2/img_001_SRF_2_LR.png"
test_dataset = SRDataset(test_hr_dir, test_lr_dir, transform, target_size=(256, 256))
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

# # 모델 로드
# model.load_state_dict(torch.load(save_path))
# model.eval()

# # PSNR과 SSIM 계산
# total_psnr = 0
# total_ssim = 0
# num_images = 0

# with torch.no_grad():
#     for lr_imgs, hr_imgs in test_loader:
#         lr_imgs, hr_imgs = lr_imgs.to(device), hr_imgs.to(device)
#         preds = model(lr_imgs).cpu().numpy().squeeze().transpose(1, 2, 0)
#         hr_imgs = hr_imgs.cpu().numpy().squeeze().transpose(1, 2, 0)

#         # PSNR 및 SSIM 계산
#         total_psnr += psnr(hr_imgs, preds, data_range=1.0)
#         total_ssim += ssim(hr_imgs, preds, data_range=1.0, multichannel=True)
#         num_images += 1

# print(f"Average PSNR: {total_psnr / num_images:.4f}, Average SSIM: {total_ssim / num_images:.4f}")


NotADirectoryError: [WinError 267] 디렉터리 이름이 올바르지 않습니다: 'C:/Users/kowm6/Desktop/SRdataset/BSD100/BSD100/image_SRF_2/img_001_SRF_2_HR.png'

In [42]:
import os
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torch.nn.functional as F  # 여기서 avg_pool2d를 사용

# PSNR 계산 함수
def calculate_psnr(img1, img2, max_val=1.0):
    mse = torch.mean((img1 - img2) ** 2)  # MSE 직접 계산
    if mse == 0:
        return float('inf')  # MSE가 0이면 무한대로 반환
    psnr = 10 * torch.log10(max_val**2 / mse)
    return psnr

# SSIM 계산 함수
def calculate_ssim(img1, img2, max_val=1.0):
    C1 = (0.01 * max_val) ** 2
    C2 = (0.03 * max_val) ** 2

    mu1 = F.avg_pool2d(img1, kernel_size=3, stride=1, padding=1)
    mu2 = F.avg_pool2d(img2, kernel_size=3, stride=1, padding=1)

    mu1_sq = mu1.pow(2)
    mu2_sq = mu2.pow(2)
    mu1_mu2 = mu1 * mu2

    sigma1_sq = F.avg_pool2d(img1 * img1, kernel_size=3, stride=1, padding=1) - mu1_sq
    sigma2_sq = F.avg_pool2d(img2 * img2, kernel_size=3, stride=1, padding=1) - mu2_sq
    sigma12 = F.avg_pool2d(img1 * img2, kernel_size=3, stride=1, padding=1) - mu1_mu2

    ssim_map = ((2 * mu1_mu2 + C1) * (2 * sigma12 + C2)) / ((mu1_sq + mu2_sq + C1) * (sigma1_sq + sigma2_sq + C2))
    return ssim_map.mean()

# 저해상도 이미지 생성 함수
def create_low_res_images(input_dir, output_dir, scale):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for filename in os.listdir(input_dir):
        if filename.endswith('.png') or filename.endswith('.jpg'):
            img = Image.open(os.path.join(input_dir, filename))
            lr_img = img.resize((img.width // scale, img.height // scale), Image.BICUBIC)
            lr_img.save(os.path.join(output_dir, filename))

# SRDataset 클래스 정의
class SRDataset(Dataset):
    def __init__(self, hr_dir, lr_dir, transform=None, target_size=(256, 256)):
        self.hr_dir = hr_dir
        self.lr_dir = lr_dir
        self.hr_filenames = sorted(os.listdir(hr_dir))
        self.lr_filenames = sorted(os.listdir(lr_dir))
        self.transform = transform
        self.target_size = target_size

    def __len__(self):
        return len(self.hr_filenames)

    def __getitem__(self, idx):
        hr_path = os.path.join(self.hr_dir, self.hr_filenames[idx])
        lr_path = os.path.join(self.lr_dir, self.lr_filenames[idx])

        hr_img = Image.open(hr_path).convert("RGB")
        lr_img = Image.open(lr_path).convert("RGB")
        hr_img = transforms.Resize(self.target_size)(hr_img)
        lr_img = transforms.Resize(self.target_size)(lr_img)

        if self.transform:
            hr_img = self.transform(hr_img)
            lr_img = self.transform(lr_img)

        return lr_img, hr_img

# SRCNN 모델 정의
class SRCNN(nn.Module):
    def __init__(self):
        super(SRCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=9, padding=4)
        self.conv2 = nn.Conv2d(64, 32, kernel_size=5, padding=2)
        self.conv3 = nn.Conv2d(32, 3, kernel_size=5, padding=2)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.conv3(x)
        return x

# 데이터 경로
train_hr_dir = r"C:\Users\kowm6\Desktop\SRdataset\DIV2K\DIV2K_train_HR"
train_lr_dir = r"C:\Users\kowm6\Desktop\SRdataset\DIV2K\DIV2K_train_LR"
valid_hr_dir = r"C:\Users\kowm6\Desktop\SRdataset\DIV2K\DIV2K_valid_HR"
valid_lr_dir = r"C:\Users\kowm6\Desktop\SRdataset\DIV2K\DIV2K_valid_LR"

# 데이터셋 및 DataLoader 설정
transform = transforms.Compose([transforms.ToTensor()])
train_dataset = SRDataset(train_hr_dir, train_lr_dir, transform, target_size=(256, 256))
valid_dataset = SRDataset(valid_hr_dir, valid_lr_dir, transform, target_size=(256, 256))
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=16, shuffle=False)

# 학습 준비
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SRCNN().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# 학습 및 검증 루프
num_epochs = 2
best_valid_loss = float('inf')
save_path = r"C:\Users\kowm6\Desktop\sr\best_model.pth"

for epoch in range(num_epochs):
    # 학습 루프
    model.train()
    train_loss = 0
    train_psnr = 0
    for lr_imgs, hr_imgs in train_loader:
        lr_imgs, hr_imgs = lr_imgs.to(device), hr_imgs.to(device)
        preds = model(lr_imgs)
        loss = criterion(preds, hr_imgs)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

        # PSNR 계산
        train_psnr += calculate_psnr(preds, hr_imgs)

    train_psnr /= len(train_loader)

    # 검증 루프
    model.eval()
    valid_loss = 0
    valid_psnr = 0
    valid_ssim = 0
    with torch.no_grad():
        for lr_imgs, hr_imgs in valid_loader:
            lr_imgs, hr_imgs = lr_imgs.to(device), hr_imgs.to(device)
            preds = model(lr_imgs)
            valid_loss += criterion(preds, hr_imgs).item()

            # PSNR 및 SSIM 계산
            valid_psnr += calculate_psnr(preds, hr_imgs)
            valid_ssim += calculate_ssim(preds, hr_imgs)

    valid_psnr /= len(valid_loader)
    valid_ssim /= len(valid_loader)
    valid_loss /= len(valid_loader)

    print(f"Epoch {epoch + 1}/{num_epochs}, Train Loss: {train_loss / len(train_loader):.4f}, "
          f"Validation Loss: {valid_loss:.4f}, Train PSNR: {train_psnr:.4f}, "
          f"Validation PSNR: {valid_psnr:.4f}, Validation SSIM: {valid_ssim:.4f}")

    # 최적 가중치 저장
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), save_path)
        print("Best model saved!")

# 테스트 데이터 평가
test_hr_dir = r"C:/Users/kowm6/Desktop/testlr"
test_lr_dir = r"C:/Users/kowm6/Desktop/testhr"
test_dataset = SRDataset(test_hr_dir, test_lr_dir, transform, target_size=(256, 256))
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

model.load_state_dict(torch.load(save_path))
model.eval()

test_psnr = 0
test_ssim = 0
num_test_images = 0

with torch.no_grad():
    for lr_imgs, hr_imgs in test_loader:
        lr_imgs, hr_imgs = lr_imgs.to(device), hr_imgs.to(device)
        preds = model(lr_imgs)

        test_psnr += calculate_psnr(preds, hr_imgs)
        test_ssim += calculate_ssim(preds, hr_imgs)
        num_test_images += 1

test_psnr /= num_test_images
test_ssim /= num_test_images

print(f"Test PSNR: {test_psnr:.4f}, Test SSIM: {test_ssim:.4f}")


Epoch 1/2, Train Loss: 0.0545, Validation Loss: 0.0143, Train PSNR: 14.8707, Validation PSNR: 18.5539, Validation SSIM: 0.5403
Best model saved!
Epoch 2/2, Train Loss: 0.0094, Validation Loss: 0.0078, Train PSNR: 20.3741, Validation PSNR: 21.1631, Validation SSIM: 0.6485
Best model saved!
Test PSNR: 22.8266, Test SSIM: 0.7084


C:\Users\kowm6\AppData\Local\Temp\ipykernel_1884\2967394493.py:169: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(save_path))


In [41]:
# 테스트 데이터 평가
test_hr_dir = r"C:/Users/kowm6/Desktop/testlr"
test_lr_dir = r"C:/Users/kowm6/Desktop/testhr"
test_dataset = SRDataset(test_hr_dir, test_lr_dir, transform, target_size=(256, 256))
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

model.load_state_dict(torch.load(save_path))
model.eval()

test_psnr = 0
test_ssim = 0
num_test_images = 0

with torch.no_grad():
    for lr_imgs, hr_imgs in test_loader:
        lr_imgs, hr_imgs = lr_imgs.to(device), hr_imgs.to(device)
        preds = model(lr_imgs)

        test_psnr += calculate_psnr(preds, hr_imgs)
        test_ssim += calculate_ssim(preds, hr_imgs)
        num_test_images += 1

test_psnr /= num_test_images
test_ssim /= num_test_images

print(f"Test PSNR: {test_psnr:.4f}, Test SSIM: {test_ssim:.4f}")


Test PSNR: 24.4085, Test SSIM: 0.7800


C:\Users\kowm6\AppData\Local\Temp\ipykernel_1884\4035062227.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(save_path))
